In [ ]:
from kid233.stdimp import *
from citkid.prima_readout.generator import generate_data
from citkid.prima_readout.create_cal import make_x_cal
from citkid.prima_readout.apply_cal import convert_iq_to_x
from citkid.prima_readout.data_io import *
from citkid.res.gain import remove_gain
from citkid.prima_readout.cosmic_rays import remove_cosmic_rays
from citkid.prima_readout.average import average_x

## Generate random data 
The following function generates fine scan data (ffine, zfine), gain scan data (fgain, zgain), and a timestream (f0 is the tone frequency and znoise is the timestream). The fine and gain scans have random parameters, and the noise timestream has fixed parameters (I want to vary these eventually), but scaled to the IQ loop. noise_factor is used to increase or decrease the average amount of noise in the IQ loops. 

In [ ]:
ffine, zfine, fgain, zgain, f0, znoise = generate_data(noise_factor = 0)

## Create and save calibration file
The following code is run on the ground to create calibration files, given the data that is measured (or in this case generated).

In [ ]:
# Create calibration data and plots
fs = [np.mean(ffine)]
Qs = [100000] # This scales with resonator Q
p_amp, p_phase, origin, v, theta_fine, p_x, figs =\
make_x_cal(ffine, zfine, fgain, zgain, fs, Qs, plotq = True)

# Save data and plots
data_directory = 'C:/data/citkid_dev/prima_readout/test/'
plot_directory = data_directory + 'plots/'
filename = 'test'
save_x_cal(data_directory, filename, p_amp, p_phase, origin, v, p_x, figs, 
           plot_directory = plot_directory, make_directories = True)
ax = figs[1].axes[0]

## Load calibration file

In [ ]:
p_amp, p_phase, origin, v, p_x = load_x_cal(data_directory + filename + '.npz')

## Convert znoise to x using the loaded calibration

In [ ]:
x = convert_iq_to_x(f0, znoise, p_amp, p_phase, origin, v, p_x)

## Remove cosmic rays
We would characterize the time constant of each detector ahead of time. We have to characterize the spread accross the array: maybe this can be a fixed value for the whole array. cr_nstd will also be characterized ahead of time. I suspect it will depend on the noise of each detector. We need to do some work on cosmic ray generator, because they look a little weird right now. There is also some work to do on the peak finding algorithm parameters.

In [ ]:
fsample = 10e3 # 10 kHz 
tsample = 1 / fsample
cr_indices, x_rmvd = remove_cosmic_rays(x, tsample, cr_nstd = 6, time_constant = 1e-3)

## Average data to 500 Hz

In [ ]:
navg = 10000 // 500
x_avg = average_x(x_rmvd, navg)

## Plot cosmic ray removal and averaging

In [ ]:
# Plot 
fig, ax = plt.subplots(figsize = [6, 2], dpi = 150, layout = 'tight')
ax.set(xlabel = 'Time (s)', ylabel = 'x (Hz / MHz)')
t = np.arange(0, tsample * len(x), tsample)
ax.plot(t, x, color = plt.cm.Greys(0.5))
ax.plot(t[cr_indices], x[cr_indices], 'xr')
ax.plot(t, x_rmvd, color = 'b')
t_avg = np.arange(0, len(x_avg) * tsample * navg, tsample * navg)
ax.plot(t_avg, x_avg, color = 'r')